<a href="https://colab.research.google.com/github/nicolas998/WMF/blob/ghos_topo/Examples/Little_siux_nicolas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load packages 

In [ ]:
!apt-get install -qq libgdal-dev libproj-dev
!pip install cartopy
!pip install netCDF4
!pip install rasterio
!pip install git+https://github.com/nicolas998/WMF.git@ghos_topo --upgrade
!pip install pysheds
!pip install geopandas 
!pip install geemap|

In [ ]:
!pip install git+https://github.com/nicolas998/WMF.git@ghos_topo --upgrade

In [ ]:
import pylab as pl 
from wmf import wmf 
import numpy as np 
import pandas as pd 
import os
import osgeo
from wmf import ghost_topo as go 
import ee
import folium
import geopandas as geop
import warnings
warnings.filterwarnings('ignore')

# Define Watershed 

## Load data 

In [3]:
DEM, epsg = wmf.read_map_raster('/content/drive/MyDrive/IIHR/2021_IWA/data/littlesiouxV2.tif', isDEMorDIR=True, dxp=90, noDataP=-9999) 
DIR, epsg = wmf.read_map_raster('/content/drive/MyDrive/IIHR/2021_IWA/data/LS90_dir.tif', isDEMorDIR=True, dxp=90, noDataP=-9999, isDIR=True) 


## Define Watershed

In [12]:
st = wmf.Stream(310015,4769947, DEM, DIR)
st.Save_Stream2Map('/content/drive/MyDrive/IIHR/2021_IWA/little_siux/guide_stream.shp', EPSG = int(epsg))
cu = wmf.SimuBasin(267781,4702029, DEM, DIR, stream=st, threshold=1000)
cu.Save_Basin2Map('/content/drive/MyDrive/IIHR/2021_IWA/little_siux/watershed.shp', dx = 90, EPSG = int(epsg), )

1


In [4]:
st = wmf.Stream(315722,4829408, DEM, DIR)
#st.Save_Stream2Map('/content/drive/MyDrive/IIHR/2021_IWA/little_siux/guide_stream.shp', EPSG = int(epsg))
cu = wmf.SimuBasin(319337,4803138, DEM, DIR, stream=st, threshold=1000)
#cu.Save_Basin2Map('/content/drive/MyDrive/IIHR/2021_IWA/little_siux/watershed.shp', dx = 90, EPSG = int(epsg), )

1


# Ghost pre-processing 

## Define mesh and river segments 

In [ ]:
gp = go.ghost_preprocess(cu, path_dem='/content/drive/MyDrive/IIHR/2021_IWA/data/littlesiouxV2.tif', 
    seg_threshold=800,
    seg_point_distance=75)

corrected = gp.get_segments_topology(epsilon=0.1)
gp.get_mesh_river_points(clean_close_points=True,
                         min_river2river_distance=50)
gp.get_mesh_grid_points(mesh_spaces=12,
                        border_iter=3,
                        clean_with_river=True,
                        min_dem2river_distance=50)

gp.get_voronoi_polygons()
gp.define_polygons_topology()

In [6]:
gp.write_mesh_file('/content/drive/MyDrive/IIHR/2021_IWA/data/mesh2.mesh','/content/drive/MyDrive/IIHR/2021_IWA/data/mesh2.shp',)
gp.write_river_file(path = '/content/drive/MyDrive/IIHR/2021_IWA/data/exampl2.riv', 
              shp_path = '/content/drive/MyDrive/IIHR/2021_IWA/data/example_riv2.shp',)

## Get soil and land use data from EE

In [ ]:
#Gets the soil data
soils = go.get_soils_data()
gp.get_physical_prop(soils, 
                     prop_name='soils', 
                     sliced=True, 
                     xdivisions=2, 
                     ydivisions=2)
#Fill the geometries with null values.
gp.polygons_shp['soils'].fillna(gp.polygons_shp['soils'].value_counts().idxmax(), 
                                inplace = True)
#Make the variable an integer
gp.polygons_shp['soils'] = gp.polygons_shp['soils'].astype(int)

In [ ]:
#Gets the land use data
land_use = go.get_land_use_data()
gp.get_physical_prop(land_use, 
                     prop_name='land', 
                     band = 'cropland',
                     sliced=True, 
                     xdivisions=2, 
                     ydivisions=2)
#Make the vartiable and integer
gp.polygons_shp['land'] = gp.polygons_shp['land'].astype(int)

## Writes the attribute file 

In [10]:
gp.polygons_shp.head()

,polygon,area[m2],z[m],nfaces,geometry,soils,land
0,1,157079.008789,1386.189148,7,"POLYGON ((318967.885 4803611.603, 319408.257 4...",7,1
1,2,229234.270874,1390.207275,6,"POLYGON ((318966.303 4803612.605, 318739.559 4...",7,176
2,3,383845.358276,1389.457886,5,"POLYGON ((318780.531 4804891.985, 319412.102 4...",7,175
3,4,334246.502319,1386.305664,6,"POLYGON ((318766.076 4805333.605, 319323.061 4...",7,176
4,5,365964.567383,1394.131042,5,"POLYGON ((318491.127 4805725.000, 318633.485 4...",7,1


In [11]:
gp.write_attribute_file('/content/drive/MyDrive/IIHR/2021_IWA/little_siux/attribute.att', 
                        'soils', 
                        'land')